In [1]:
import transcriptic 
from transcriptic import commands, Container
from transcriptic.config import Connection
from transcriptic.jupyter import objects
import json
from pysd2cat.data import tx_fcs
from pysd2cat.data import pipeline
from pysd2cat.analysis import biofab_live_dead_analysis as blda
from pysd2cat.data import biofab_live_dead as bld


from os.path import expanduser
import requests
import os
import zipfile
import shutil
import fnmatch
import glob
import pandas as pd
import ast

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

Connection.from_file("~/.transcriptic")

/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
#protocol = objects.protocol('pr1drp6hzrchh79')
[x['name'] for x in transcriptic.api.get_protocols() if x['id'] == 'pr1drp6hzrchh79']

[]

In [2]:
container_string = 'ct1drnsvwmpex75' #'422936' #
results = transcriptic.api.inventory(container_string)


In [ ]:
import types
def imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            yield val.__name__
list(imports())

In [4]:
results['results']

[]

In [3]:
len([x['id'] for x in results['results']])

8

In [79]:
c = objects.Container(results['results'][0]['id'])

In [82]:
c.container_type.well_count

96

In [6]:
#c.aliquots.loc[c.aliquots['Name'] == 'UWBF_25784']
container_strains = c.aliquots.Name.unique()
sample_strains = set([ 'UWBF_24952', 'UWBF_24959', 'UWBF_24960',
       'UWBF_24961', 'UWBF_24962', 'UWBF_25784', 'UWBF_24864',
       'UWBF_23970', 'UWBF_6390'])
sample_strains.issubset(container_strains)
#col = 1
#c.aliquots.loc[(c.aliquots.index + col) % 12  == 0]

True

In [123]:
def get_matching_aliquots(strains, container_id, merge_key="Name"):
    c = objects.Container(container_id)
    strain_df = pd.DataFrame({merge_key : strains})
    c.aliquots['index'] = c.aliquots.index
    matches = c.aliquots.merge(strain_df, on=merge_key, how="inner")
    matches = matches.set_index('index')
    
    print("Matched:\n", str(matches.groupby(["Name"]).agg(len)))
    print("Didn't Match:\n", str([x for x in strains if x not in matches["Name"].unique()]))
    return matches
    
def get_src_wells_from_aliquots(container_id, matching_aliquots):
    print(matching_aliquots)
    return [ "{}/{}".format(container_id, x) for x in matching_aliquots.index.tolist() ]
    
def get_src_wells_matching_aliquots(strains, container_search_string=""):
    results = transcriptic.api.inventory(container_search_string)
    for result in results['results']:
        container_id = result['id']
        matching_aliquots = get_matching_aliquots(strains, container_id)
        if matching_aliquots is not None:
            src_wells = get_src_wells_from_aliquots(container_id, matching_aliquots)
            return src_wells
    return None
    


strains = [ 'UWBF_24926', 'UWBF_24952', 'UWBF_24959', 'UWBF_24960', 'UWBF_24961', 'UWBF_24962', 'UWBF_25784', 'UWBF_23970', 'UWBF_6390', 'UWBF_24864' ]
#strains = [  'UWBF_24952']

get_src_wells_matching_aliquots(strains, container_search_string=container_string)

Matched:
             Id  Volume  SBH_URI
Name                           
UWBF_23970  3   3       3      
UWBF_24864  3   3       3      
UWBF_24926  12  12      12     
UWBF_24952  12  12      12     
UWBF_24959  12  12      12     
UWBF_24960  12  12      12     
UWBF_24961  12  12      12     
UWBF_24962  12  12      12     
UWBF_25784  12  12      12     
UWBF_6390   3   3       3      
Didn't Match:
 []
             Name               Id         Volume                                             SBH_URI
index                                                                                                
0      UWBF_24926  aq1dqvp6ntrkkp5  40:microliter  https://hub.sd2e.org/user/sd2e/design/UWBF_24926/1
1      UWBF_24926  aq1dqvp6ntsqst6  40:microliter  https://hub.sd2e.org/user/sd2e/design/UWBF_24926/1
2      UWBF_24926  aq1dqvp6nttuyx7  40:microliter  https://hub.sd2e.org/user/sd2e/design/UWBF_24926/1
3      UWBF_24926  aq1dqvp6nttuyx8  40:microliter  https://hub.sd2e.org/user/s

['ct1dqvp6ntpb7e3/0',
 'ct1dqvp6ntpb7e3/1',
 'ct1dqvp6ntpb7e3/2',
 'ct1dqvp6ntpb7e3/3',
 'ct1dqvp6ntpb7e3/4',
 'ct1dqvp6ntpb7e3/5',
 'ct1dqvp6ntpb7e3/6',
 'ct1dqvp6ntpb7e3/7',
 'ct1dqvp6ntpb7e3/8',
 'ct1dqvp6ntpb7e3/9',
 'ct1dqvp6ntpb7e3/10',
 'ct1dqvp6ntpb7e3/11',
 'ct1dqvp6ntpb7e3/12',
 'ct1dqvp6ntpb7e3/13',
 'ct1dqvp6ntpb7e3/14',
 'ct1dqvp6ntpb7e3/15',
 'ct1dqvp6ntpb7e3/16',
 'ct1dqvp6ntpb7e3/17',
 'ct1dqvp6ntpb7e3/18',
 'ct1dqvp6ntpb7e3/19',
 'ct1dqvp6ntpb7e3/20',
 'ct1dqvp6ntpb7e3/21',
 'ct1dqvp6ntpb7e3/22',
 'ct1dqvp6ntpb7e3/23',
 'ct1dqvp6ntpb7e3/24',
 'ct1dqvp6ntpb7e3/25',
 'ct1dqvp6ntpb7e3/26',
 'ct1dqvp6ntpb7e3/27',
 'ct1dqvp6ntpb7e3/28',
 'ct1dqvp6ntpb7e3/29',
 'ct1dqvp6ntpb7e3/30',
 'ct1dqvp6ntpb7e3/31',
 'ct1dqvp6ntpb7e3/32',
 'ct1dqvp6ntpb7e3/33',
 'ct1dqvp6ntpb7e3/34',
 'ct1dqvp6ntpb7e3/35',
 'ct1dqvp6ntpb7e3/36',
 'ct1dqvp6ntpb7e3/37',
 'ct1dqvp6ntpb7e3/38',
 'ct1dqvp6ntpb7e3/39',
 'ct1dqvp6ntpb7e3/40',
 'ct1dqvp6ntpb7e3/41',
 'ct1dqvp6ntpb7e3/42',
 'ct1dqvp6ntpb7e3/43'

In [109]:
c = objects.Container("ct1dqvp6ntpb7e3")

In [115]:
c.

SyntaxError: invalid syntax (<ipython-input-115-cf463a5954f9>, line 1)

In [21]:
def get_design_df(design_file):
    j = json.load(open(design_file, 'r'))
    return pd.read_json(j['design'])
    
print(os.getcwd())
this_dir = os.getcwd()
#df = get_design_df(os.path.realpath(os.path.join(this_dir, '../../../tmp/YEAST_STATES_GROWTH_CURVE/experiments/growth_curve_test1/design_growth_curve_test1.json')))
#df = get_design_df(os.path.realpath(os.path.join(this_dir, '../../../tmp/YEAST_STATES_GROWTH_CURVE/experiments/growth_curve_test1_corrected/design_growth_curve_test1_corrected.json')))
#df = get_design_df(os.path.realpath(os.path.join(this_dir, '../../../tmp/YEAST_STATES/experiments/dose_response_test1/design_dose_response_test1.json')))
#df = get_design_df(os.path.realpath(os.path.join(this_dir, '../../../tmp/YEAST_STATES/experiments/dose_response_test1/design_dose_response_test1.json')))
#df = get_design_df(os.path.realpath(os.path.join(this_dir, '../../../tmp/YEAST_STATES/experiments/YeastSTATES CRISPR Long Duration Time Series Request (20191208)/design_YeastSTATES CRISPR Long Duration Time Series Request (20191208).json')))
#df = get_design_df(os.path.realpath(os.path.join(this_dir, '../../../tmp/NOVEL_CHASSIS/experiments/CP_NovelChassis_OR_Circuit_Cycle0/design_CP_NovelChassis_OR_Circuit_Cycle0.json')))
df = get_design_df(os.path.realpath(os.path.join(this_dir, '../../../tmp/YEAST_STATES_OBSTACLE_COURSE/experiments/Obstacle_course1/design_Obstacle_course1.json')))
df

/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/pysd2cat/notebooks


,Dilution factor,be@18,be@42,column_id,exp_info.growth_volumes.culture_volume,exp_info.overnight_volumes.culture_volume,exp_info.overnight_volumes.media_volume,exp_info.sample_time,lab,measurement_info.36_hr_read,measurement_info.flow_info,measurement_info.plate_reader_info,media,protocol,reagent_info.base_media,reagent_info.inducer_info,reagent_info.kill_switch,replicate,sc@18,sc@42,strain,temperature,batch,id,experiment_id,measurement_type,timepoint,output_id,container
0,51,on,off,1,5:microliter,10:microliter,990:microliter,8:hour,Strateos,False,"{'do_flow': {'value': 'every-sampling', 'inputs': {'every-sampling': {'flow_params': 'yeast'}}}}","[{'read_choice': {'value': 'fluorescence', 'inputs': {'fluorescence': {'excitation': '488:nanometer', 'emission': '520:nanometer', 'gain': 0.16}}}}, {'read_choice': {'value': 'luminescence', 'inputs': {'luminescence': {'integration_time': '1:second'}}}}]",SC Media,obstacle_course,SC Media,"{'inputs': {'2': {'conditions': {'inputs': {'1': {'condition': [{'day_1': 'ab'}]}}, 'value': '1'}, 'inducer_1': {'containerId': 'ct1dvadszwgqwr6', 'wellIndex': 0}, 'inducer_1_final_conc': {'magnitude': 0.005, 'unit': 'millimolar'}, 'inducer_2': {'containerId': 'ct1dvadszwtxpwf', 'wellIndex': 0}, 'inducer_2_final_conc': {'magnitude': 0.05, 'unit': 'millimolar'}}}, 'value': '2'}","{'value': 'false', 'inputs': {'false': {}}}",1,off,on,UWBF_6390,30,0,Obstacle_course1_0_dc561f08-3624-11ea-acd6-0242ac110002,Obstacle_course1,PLATE_READER,26,0,ct1dv7kqg9899q4
1,51,on,off,1,5:microliter,10:microliter,990:microliter,8:hour,Strateos,False,"{'do_flow': {'value': 'every-sampling', 'inputs': {'every-sampling': {'flow_params': 'yeast'}}}}","[{'read_choice': {'value': 'fluorescence', 'inputs': {'fluorescence': {'excitation': '488:nanometer', 'emission': '520:nanometer', 'gain': 0.16}}}}, {'read_choice': {'value': 'luminescence', 'inputs': {'luminescence': {'integration_time': '1:second'}}}}]",SC Media,obstacle_course,SC Media,"{'inputs': {'2': {'conditions': {'inputs': {'1': {'condition': [{'day_1': 'ab'}]}}, 'value': '1'}, 'inducer_1': {'containerId': 'ct1dvadszwgqwr6', 'wellIndex': 0}, 'inducer_1_final_conc': {'magnitude': 0.005, 'unit': 'millimolar'}, 'inducer_2': {'containerId': 'ct1dvadszwtxpwf', 'wellIndex': 0}, 'inducer_2_final_conc': {'magnitude': 0.05, 'unit': 'millimolar'}}}, 'value': '2'}","{'value': 'false', 'inputs': {'false': {}}}",3,off,on,UWBF_6390,30,0,Obstacle_course1_0_dc562f66-3624-11ea-acd6-0242ac110002,Obstacle_course1,PLATE_READER,26,12,ct1dv7kqg9899q4
2,51,on,off,1,5:microliter,10:microliter,990:microliter,8:hour,Strateos,False,"{'do_flow': {'value': 'every-sampling', 'inputs': {'every-sampling': {'flow_params': 'yeast'}}}}","[{'read_choice': {'value': 'fluorescence', 'inputs': {'fluorescence': {'excitation': '488:nanometer', 'emission': '520:nanometer', 'gain': 0.16}}}}, {'read_choice': {'value': 'luminescence', 'inputs': {'luminescence': {'integration_time': '1:second'}}}}]",SC Media,obstacle_course,SC Media,"{'inputs': {'2': {'conditions': {'inputs': {'1': {'condition': [{'day_1': 'ab'}]}}, 'value': '1'}, 'inducer_1': {'containerId': 'ct1dvadszwgqwr6', 'wellIndex': 0}, 'inducer_1_final_conc': {'magnitude': 0.005, 'unit': 'millimolar'}, 'inducer_2': {'containerId': 'ct1dvadszwtxpwf', 'wellIndex': 0}, 'inducer_2_final_conc': {'magnitude': 0.05, 'unit': 'millimolar'}}}, 'value': '2'}","{'value': 'false', 'inputs': {'false': {}}}",2,off,on,UWBF_6390,30,0,Obstacle_course1_0_dc563542-3624-11ea-acd6-0242ac110002,Obstacle_course1,PLATE_READER,26,24,ct1dv7kqg9899q4
3,51,on,off,1,5:microliter,10:microliter,990:microliter,8:hour,Strateos,False,"{'do_flow': {'value': 'every-sampling', 'inputs': {'every-sampling': {'flow_params': 'yeast'}}}}","[{'read_choice': {'value': 'fluorescence', 'inputs': {'fluorescence': {'excitation': '488:nanometer', 'emission': '520:nanometer', 'gain': 0.16}}}}, {'read_choice': {'value': 'luminescence', 'inputs': {'luminescence': {'integration_time': '1:second'}}}}]",SC Medi

In [18]:
#df.lab_id
#f.loc[df.batch==1] #.drop(columns='id').drop_duplicates()
#df.drop(columns='timepoint').drop_duplicates().groupby([  "strain"]).agg(len)
#df.drop(columns=['timepoint', 'measurement_type']).drop_duplicates().groupby([ 'strain',  'beta_estradiol_concentration', 'batch']).agg(len).reset_index()
#df.drop(columns=['timepoint', 'measurement_type']).drop_duplicates().groupby([ 'strain',  'batch']).agg(len).reset_index()
df[['column_id', 'dilution1@18', 'dilution2@18']].drop_duplicates()
#df.loc[df['beta_estradiol_concentration'].isna()].strain.value_counts()
#df.strain.unique()
#df.lab_id.unique()
#df.measurement_type.unique()
#df.columns
#len(df.drop(columns=['timepoint', 'measurement_type']).drop_duplicates().loc[df.strain == 'UWBF_24926'])
#len(df.loc[df.strain == 'UWBF_24952'])

,column_id,dilution1@18,dilution2@18
0,1,off,on
8,2,off,off
16,3,on,on
24,4,off,off
32,5,off,on
40,6,on,on
48,7,off,off
56,8,on,off
64,9,on,off
72,10,off,on


In [87]:
#df.loc[(df.timepoint==8.0) & (df.iptg == 0) & (df.cuminic_acid == 0) & (df.temperature ==30) & (df.colony==1) & (df.strain == "https://hub.sd2e.org/user/sd2e/design/B_subtilis_GFP_min/1")]
df.loc[df.strain == ""].output_id.unique()

array([84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95])

In [26]:
from pysmt.shortcuts import Symbol, And, Not, is_sat, get_model

varA = Symbol("A") # Default type is Boolean
varB = Symbol("B")
f = And(varA, Not(varB))
model = get_model(f)
print(model)

A := True
B := False


In [19]:
from pysd2cat.analysis.plate_layout import solve

from pysmt.shortcuts import Symbol, And, Or, Not, Implies, Equals, Iff, is_sat, get_model, GT, GE, LT, Int, String
from pysmt.typing import INT, StringType


## Symbols denoting the samples
samples = ["x1", "x2", "x3", "x4"]

## Factor definitions
factors = { 
    "media" : {
        "domain" : ['m1', "m2"],
        "ftype" : "batch"
    }, 
    "strain" : {
        "domain" : [ "s1", "s2"],
        "ftype" : "sample"
    },
    "temperature" : {
        "domain" : [ "30", "35"],
        "ftype" : "experiment"
    },
    "measurement_type" : {
        "domain" : [ "PLATE_READER", "FLOW"],
        "ftype" : "shadow"
    },
    "inducer" : {
        "domain" : [ "0", "1"],
        "ftype" : "column"
    }
}


containers = { "c1" : {
                    "aliquots" : {
                        "a1" : {
                            "strain" : "s1"
                        },
                        "a2" : {
                            "strain" : "s2"
                        } 
                    },
                    "columns" : { "col1" : ["a1", "a2"] }
                } 
             }



## Requirement for the samples
requirements = [
    {
        "factors" : [
            {
                "factor" : "media", "values" : [ "m2" ]
            },
            {
                "factor" : "strain", "values" : [ "s1", "s2" ]
            },
            {
                "factor" : "temperature", "values" : [ "30" ]
            },
            {
                "factor" : "measurement_type", "values" : [ "PLATE_READER", "FLOW" ]
            },
            {
                "factor" : "inducer", "values" : [  "1" ]
            }
        ]
    }
]

inputs = {
    "samples" : samples,
    "factors" : factors,
    "containers" : containers,
    "requirements" : requirements
}


model = solve(inputs)
if model:
    print(model)
else:
    print("No solution found")

'media(x1)' := 1
'inducer(x4)' := 1
'strain(x1)' := 1
'inducer(x2)' := 1
'measurement_type(x4)' := 1
'inducer(x3)' := 1
'inducer(x1)' := 1
'strain(x4)' := 1
'temperature(x4)' := 0
'media(x4)' := 1
'media(x3)' := 1
'temperature(x3)' := 0
'strain(x2)' := 0
'measurement_type(x1)' := 0
'temperature(x1)' := 0
'measurement_type(x3)' := 1
'temperature(x2)' := 0
media_c1_batch := 1
'measurement_type(x2)' := 0
'strain(x3)' := 0
tau_x1 := 1
tau_x3 := 0
'media(x2)' := 1
temperature_exp := 0
tau_x2 := 0
tau_x4 := 1


In [2]:
df = pd.read_csv("../dan.csv")
df

,Unnamed: 0,aliquot,container,replicate,strain,sample,measurement_type,timepoint,recovery_media,temperature
0,0,a1,422936.1,1,UWBF_24959,x0_a1_422936.1,PLATE_READER,9.0,sc_media,35.0
1,1,a1,422936.1,1,UWBF_24959,x1_a1_422936.1,PLATE_READER,12.0,sc_media,35.0
2,2,a1,422936.1,1,UWBF_24959,x2_a1_422936.1,FLOW,3.0,sc_media,35.0
3,3,a1,422936.1,1,UWBF_24959,x3_a1_422936.1,FLOW,1.0,sc_media,35.0
4,4,a1,422936.1,1,UWBF_24959,x4_a1_422936.1,PLATE_READER,6.0,sc_media,35.0
...,...,...,...,...,...,...,...,...,...,...
1675,1675,g9,422936.2,9,UWBF_25784,x5_g9_422936.2,FLOW,12.0,sc_media,30.0
1676,1676,g9,422936.2,9,UWBF_25784,x6_g9_422936.2,FLOW,1.0,sc_media,30.0
1677,1677,g9,422936.2,9,UWBF_25784,x7_g9_422936.2,FLOW,6.0,sc_media,30.0
1678,1678,g9,422936.2,9,UWBF_25784,x8_g9_422936.2,PLATE_READER,12.0,sc_media,30.0


In [15]:
df.loc[df.aliquot.str.contains('a2')].sort_values(by=['temperature', 'replicate'])

,Unnamed: 0,aliquot,container,replicate,strain,sample,measurement_type,timepoint,recovery_media,temperature
352,352,a2,422936.2,1,UWBF_24959,x0_a2_422936.2,PLATE_READER,3,sc_media,30.0
353,353,a2,422936.2,1,UWBF_24959,x1_a2_422936.2,PLATE_READER,1,sc_media,30.0
354,354,a2,422936.2,1,UWBF_24959,x2_a2_422936.2,FLOW,1,sc_media,30.0
355,355,a2,422936.2,1,UWBF_24959,x3_a2_422936.2,FLOW,3,sc_media,30.0
16,16,a2,422936.1,7,UWBF_24959,x0_a2_422936.1,PLATE_READER,3,sc_media,35.0
17,17,a2,422936.1,7,UWBF_24959,x1_a2_422936.1,FLOW,3,sc_media,35.0
18,18,a2,422936.1,7,UWBF_24959,x2_a2_422936.1,PLATE_READER,1,sc_media,35.0
19,19,a2,422936.1,7,UWBF_24959,x3_a2_422936.1,FLOW,1,sc_media,35.0
